# 🎨 Imagen API로 나만의 AI 아트 만들기

안녕하세요! 텍스트 설명만으로 놀라운 이미지를 만들어내는 Google의 Imagen 모델 사용법을 배우는 것에 오신 것을 환영합니다. 🎉

이 Colab 노트북을 통해, 여러분은 간단한 Python 코드로 고품질 이미지를 생성하는 방법을 배우게 됩니다. 상상하는 모든 것을 시각적인 현실로 만들어 보세요. "붉은 스케이트보드를 든 로봇"부터 "인상주의 화풍으로 그린 풍력 발전소"까지, 가능성은 무한합니다.

준비되셨나요? 이제 여러분의 창의력을 발휘할 시간입니다!

## 1단계: API 키 준비 및 라이브러리 설치

텍스트를 생성했던 Gemini API와 동일한 환경을 사용합니다. API 키가 이미 준비되었다고 가정하고, 먼저 필요한 라이브러리를 설치하겠습니다.

In [ ]:
# Google 생성형 AI SDK를 설치합니다.
!pip install -q -U google-genai

## 2단계: API 키 설정하기 (Colab에서 안전하게)

Google Colab의 '보안 비밀' 기능을 사용하여 API 키를 안전하게 설정합니다.

1.  Colab 왼쪽 메뉴에서 **열쇠 모양(🔑) 아이콘**을 클릭하세요.
2.  **`+ 새 보안 비밀 추가`**를 누르세요.
3.  **이름**에는 `GEMINI_API_KEY`를, **값**에는 발급받은 API 키를 붙여넣으세요.
4.  **'노트북 액세스'** 스위치를 켜주세요.

아래 코드는 이렇게 저장된 키를 불러와 라이브러리를 설정하는 역할을 합니다.

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai
from google.generativeai import types
from PIL import Image
from io import BytesIO

try:
    # Colab 보안 비밀에서 API 키를 가져옵니다.
    api_key = userdata.get('GEMINI_API_KEY')
    os.environ['GEMINI_API_KEY'] = api_key
    genai.configure(api_key=api_key)
    print("API 키 설정이 완료되었습니다! ✨")
except Exception as e:
    print("🚨 API 키를 설정하는 데 실패했습니다. Colab의 '보안 비밀' 설정을 확인해주세요.")

## 3단계: 첫 번째 이미지 생성하기

이제 모든 준비가 끝났습니다. `client.models.generate_images()` 함수를 사용해 첫 이미지를 만들어 봅시다. 가장 중요한 파라미터는 `model`과 `prompt`입니다.

- **`model`**: 사용할 Imagen 모델의 이름을 지정합니다. (예: `imagen-4.0-generate-001`)
- **`prompt`**: 만들고 싶은 이미지에 대한 텍스트 설명입니다. 영어로 작성해야 합니다.

In [ ]:
# genai 클라이언트를 초기화합니다.
client = genai.Client()

# 이미지 생성을 요청합니다.
response = client.models.generate_images(
    model='imagen-4.0-generate-001', # 사용할 모델
    prompt='A photorealistic image of a robot holding a red skateboard', # 이미지 설명
)

# 생성된 이미지를 화면에 표시합니다.
for img in response.generated_images:
    img.image.show()

## 4단계: 생성 옵션으로 결과물 제어하기

`config` 파라미터를 사용하면 이미지 생성 과정을 더 세밀하게 제어할 수 있습니다.

- `number_of_images`: 생성할 이미지의 개수를 1에서 4 사이로 지정합니다.
- `aspect_ratio`: 이미지의 가로세로 비율을 설정합니다. `"1:1"`, `"16:9"`, `"9:16"` 등이 가능합니다.
- `person_generation`: 사람 이미지 생성 여부를 설정합니다. 기본값은 성인만 허용하는 `"allow_adult"`입니다.

In [ ]:
response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt='A cozy cabin in the woods during a snowy night, with warm light coming from the windows.',
    config=types.GenerateImagesConfig(
        number_of_images=2,          # 이미지를 2개 생성합니다.
        aspect_ratio="16:9"          # 와이드스크린 (16:9) 비율로 설정합니다.
    )
)

for img in response.generated_images:
    img.image.show()

## 5단계: 좋은 프롬프트 작성의 기본

원하는 이미지를 얻으려면 프롬프트를 잘 작성하는 것이 매우 중요합니다. 좋은 프롬프트는 보통 다음 세 가지 요소를 포함합니다.

1.  **주제 (Subject)**: 이미지의 핵심이 되는 대상입니다. (예: a cat, a futuristic car)
2.  **배경/맥락 (Context/Background)**: 주제가 놓일 환경입니다. (예: sitting on a bookshelf, driving on a neon-lit city street)
3.  **스타일 (Style)**: 이미지의 전체적인 화풍이나 느낌입니다. (예: watercolor painting, realistic photography, cartoon style)

**예시:** `A sketch (스타일) of a modern apartment building (주제) surrounded by skyscrapers (배경)`

처음에는 간단하게 시작해서, 생성된 이미지를 보며 점차 세부사항(형용사, 특정 조명, 카메라 각도 등)을 추가해 나가는 것이 좋습니다.

## 6단계: 이미지에 텍스트 추가하기

Imagen은 이미지 안에 텍스트를 넣는 기능도 지원합니다. 프롬프트에 원하는 텍스트를 따옴표로 감싸서 명시하면 됩니다.

- **간단하게 유지하세요**: 텍스트는 짧을수록 좋습니다.
- **반복 시도하세요**: 원하는 결과가 나올 때까지 몇 번 다시 생성해야 할 수 있습니다.

In [ ]:
response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt="""A logo for a coffee shop named 'Morning Brew'. The style should be minimalist and modern.""",
    config=types.GenerateImagesConfig(number_of_images=2)
)

for img in response.generated_images:
    img.image.show()

## ✍️ 나만의 아트 갤러리 만들기 (연습 문제)

이제 여러분이 직접 프롬프트를 만들어 볼 차례입니다! 아래 문제들의 `prompt` 변수와 `config` 설정을 자유롭게 바꿔보며 멋진 이미지를 생성해보세요.

### 연습문제 1: 사실적인 동물 사진
선글라스를 쓴 골든 리트리버의 '폴라로이드 사진'을 만들어보세요.

In [ ]:
prompt_1 = """___""" # 이 부분을 채워보세요. 예: A polaroid photo of a golden retriever wearing sunglasses

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_1,
    config=types.GenerateImagesConfig(number_of_images=1, aspect_ratio="1:1")
)
response.generated_images[0].image.show()

### 연습문제 2: 스마트폰 배경화면 만들기
거대한 초고층 빌딩을 '디지털 렌더링' 스타일로 그리고, 스마트폰 배경화면에 적합하도록 세로가 긴 `9:16` 비율로 생성해보세요.

In [ ]:
prompt_2 = """___""" # 이 부분을 채워보세요.

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_2,
    config=types.GenerateImagesConfig(
        number_of_images=1,
        aspect_ratio="___" # 비율을 채워보세요.
    )
)
response.generated_images[0].image.show()

### 연습문제 3: 특정 화가 스타일 흉내내기
풍력 발전소를 '인상주의 회화' 스타일로 그려보세요.

In [ ]:
prompt_3 = """An impressionist painting of a wind farm."""

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_3
)
response.generated_images[0].image.show()

### 연습문제 4: 로고 디자인하기
'Silo'라는 텍스트를 포함하는 소프트웨어 회사의 '모던한 로고'를 단색 배경 위에 생성해보세요.

In [ ]:
prompt_4 = """___""" # 이 부분을 채워보세요.

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_4,
    config=types.GenerateImagesConfig(number_of_images=2)
)
for img in response.generated_images:
    img.image.show()

### 연습문제 5: 상상 속의 물건 만들기
'치즈로 만든 더플 백' 이미지를 생성해보세요.

In [ ]:
prompt_5 = """A duffle bag made of cheese."""

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_5
)
response.generated_images[0].image.show()

### 연습문제 6: 특정 카메라 기법 사용하기
'어안 렌즈'로 찍은 뉴욕 거리 사진을 만들어보세요.

In [ ]:
prompt_6 = """___""" # 이 부분을 채워보세요. 예: A photo of a New York street taken with a fisheye lens

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_6
)
response.generated_images[0].image.show()

### 연습문제 7: 조명 효과 주기
해변에 앉아 있는 남성의 사진을 '황금 시간(golden hour) 조명' 효과를 주어 생성해보세요. 가로가 긴 `16:9` 비율로 만들어보세요.

In [ ]:
prompt_7 = """A close-up of a man sitting on a beach, with golden hour lighting."""
config_7 = types.GenerateImagesConfig(
    number_of_images=1,
    aspect_ratio="___" # 비율을 채워보세요.
)

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_7,
    config=config_7
)
response.generated_images[0].image.show()

### 연습문제 8: 흑백 사진 만들기
피아노를 연주하는 손가락을 클로즈업한 '흑백 영화' 느낌의 사진을 만들어보세요.

In [ ]:
prompt_8 = """___""" # 이 부분을 채워보세요.

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_8
)
response.generated_images[0].image.show()

### 연습문제 9: 포스터 만들기
'Summerland'라는 제목과 'Summer never felt so good'라는 슬로건이 들어간 포스터를 만들어보세요.

In [ ]:
prompt_9 = """A poster with the title 'Summerland' in bold font, and below this text is the slogan 'Summer never felt so good'. The background is a beautiful beach scene."""

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_9
)
response.generated_images[0].image.show()

### 연습문제 10: 자유롭게 창작하기
지금까지 배운 모든 기법을 활용하여 여러분이 상상하는 가장 멋진 이미지를 만들어보세요! 주제, 스타일, 구도, 조명 등 모든 것을 자유롭게 조합해보세요.

In [ ]:
prompt_10 = """___""" # 여러분의 상상력을 마음껏 펼쳐보세요!
config_10 = types.GenerateImagesConfig(
    number_of_images=1, # 원하는 대로 수정해보세요.
    aspect_ratio="1:1" # 원하는 대로 수정해보세요.
)

response = client.models.generate_images(
    model='imagen-4.0-generate-001',
    prompt=prompt_10,
    config=config_10
)
response.generated_images[0].image.show()